In [1]:
!pip install experta
!pip install frozendict==2.3.7

  Preparing metadata (setup.py) ... done
  Created wheel for frozendict: filename=frozendict-1.2-py3-none-any.whl size=3149 sha256=ad20efc77f1123455b219c7f3bb90ea62d42b207737e187cd9f8c136c8d43924
  Stored in directory: /root/.cache/pip/wheels/49/ac/f8/cb8120244e710bdb479c86198b03c7b08c3c2d3d2bf448fd6e
Successfully built frozendict
  Attempting uninstall: frozendict
    Found existing installation: frozendict 2.4.6
    Uninstalling frozendict-2.4.6:
      Successfully uninstalled frozendict-2.4.6
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
yfinance 0.2.59 requires frozendict>=2.3.4, but you have frozendict 1.2 which is incompatible.
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 311.6/311.6 kB 6.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for frozendict: filename=frozendict-2.3.7-py3-none-any.whl size=13594 sha256=d36f8a179e5f82

In [29]:
import pandas as pd
from experta import *
import re

# Load course and policy data from CSV files
courses_df = pd.read_csv("Cyber Security_Courses.csv").fillna("")
policies_df = pd.read_csv("policies.csv").fillna("")

# Sample student input for testing
student_input = {
    "cgpa": 3.2,                      # CGPA affects how many credits the student can take
    "semester": "FALL",              # Only courses offered this semester will be considered
    "passed_courses": ["UC1"],       # Courses the student has already completed
    "failed_courses": []             # Courses the student failed (can be prioritized if allowed)
}

# Convert the course DataFrame to a list of dictionaries for easier access
all_courses = courses_df.to_dict(orient="records")

# Define the fact model (we’ll use this to inject student info into the expert system)
class StudentProfile(Fact):
    pass

# Inference engine using Experta
class AdvisingEngine(KnowledgeEngine):
    def __init__(self, courses, student_data, policies_df):
        super().__init__()
        self.courses = courses
        self.student_data = student_data
        self.policies_df = policies_df
        self.recommended_courses = []
        self.total_credits = 0
        self.credit_limit = self.get_dynamic_credit_limit()

    # This function reads the policy rules and applies the correct credit limit based on CGPA
    def get_dynamic_credit_limit(self):
        credit_policies = self.policies_df[self.policies_df["Category"].str.strip() == "Credit Limit"]
        rules = []

        # Parse each condition and turn it into a usable function
        for _, row in credit_policies.iterrows():
            condition = str(row["Condition"])
            max_credit = int(row["max"])
            match = re.findall(r'(\d+\.\d+)', condition)

            # Different formats for different condition types
            if "≥" in condition or ">=" in condition:
                rules.append((lambda cgpa, val=float(match[0]): cgpa >= val, max_credit))
            elif "≤" in condition and len(match) == 2:
                l, u = float(match[0]), float(match[1])
                rules.append((lambda cgpa, l=l, u=u: l <= cgpa < u, max_credit))
            elif "<" in condition:
                rules.append((lambda cgpa, val=float(match[0]): cgpa < val, max_credit))

        # Return the first matching rule
        for rule, limit in rules:
            if rule(self.student_data["cgpa"]):
                return limit

        return 12  # fallback in case no rule matches

    # Main logic that recommends courses
    @Rule(StudentProfile())
    def recommend_courses(self):
        already_added = set()  # Keep track of what's already recommended
        passed = [c.strip() for c in self.student_data["passed_courses"]]
        failed = [c.strip() for c in self.student_data["failed_courses"]]

        # Step 1: Prioritize failed courses if they’re eligible
        for course in self.courses:
            code = str(course["Course Code"]).strip()

            if code not in failed:
                continue

            offered = str(course["Semester Offered"]).strip().upper()
            prereqs = [p.strip() for p in str(course["Prerequisites"]).split(",") if p.strip()]
            credits = int(course["Credit Hours"])

            if code in passed:
                continue  # skip if already passed
            if self.student_data["semester"].upper() not in offered and offered != "BOTH":
                continue  # skip if not offered this term
            if any(pr not in passed for pr in prereqs):
                continue  # prerequisites not satisfied
            if self.total_credits + credits > self.credit_limit:
                continue  # would exceed the max allowed credits

            # All checks passed — recommend the course
            self.recommended_courses.append(course)
            already_added.add(code)
            self.total_credits += credits

        # Step 2: Add new courses that are eligible and don’t conflict
        for course in self.courses:
            code = str(course["Course Code"]).strip()
            if code in passed or code in already_added:
                continue

            offered = str(course["Semester Offered"]).strip().upper()
            prereqs = [p.strip() for p in str(course["Prerequisites"]).split(",") if p.strip()]
            coreqs = [c.strip() for c in str(course["Co-requisites"]).split(",") if c.strip()]
            credits = int(course["Credit Hours"])

            if self.student_data["semester"].upper() not in offered and offered != "BOTH":
                continue  # not offered this term
            if any(pr not in passed for pr in prereqs):
                continue  # missing prerequisites
            if any(cr not in passed and cr not in [c["Course Code"].strip() for c in self.recommended_courses] for cr in coreqs):
                continue  # missing co-requisites
            if self.total_credits + credits > self.credit_limit:
                continue  # credit limit would be exceeded

            # Course passes all filters — add it
            self.recommended_courses.append(course)
            already_added.add(code)
            self.total_credits += credits

# Instantiate and run the engine
engine = AdvisingEngine(all_courses, student_input, policies_df)
engine.reset()
engine.declare(StudentProfile(**student_input))
engine.run()

# Display output
recommended = pd.DataFrame(engine.recommended_courses)
if recommended.empty:
    print("No courses could be recommended based on your profile.")
else:
    print("Recommended Courses:")
    print(recommended[["Course Code", "Course Name", "Credit Hours"]])
    print(f"Total Credits: {engine.total_credits}")


Recommended Courses:
  Course Code                           Course Name  Credit Hours
0     MAT111                         Mathematics I              3
1      MAT123                             Mechanics             3
2     MEC011               Engineering Drawing (1)              3
3     PHY212   Introduction to Engineering Physics              3
4     CSE014               Structured Programming               3
5      MAT212                       Linear Algebra              3
6     CSE131                         Logic Design               3
Total Credits: 21
